# Create an Entangled Bell Pair
We build and run the following circuit using a single-qubit Hadamard gate (denoted as ```H```) acting on the first qubit followed by a two-qubit ```CNOT``` gate: 
![](./bell_circuit.png)

# Setup S3 Bucket

In [24]:
import boto3

region = boto3.Session().region_name
aws_account_id = boto3.client("sts").get_caller_identity()["Account"]

In [2]:
!aws s3 mb s3://amazon-braket-{aws_account_id}

make_bucket: amazon-braket-835319576252


In [3]:
s3_output_prefix = (f"amazon-braket-{aws_account_id}", "bell-output")

# Setup Quantum Device

In [12]:
from braket.aws import AwsDevice

#device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")

In [13]:
print(device)

Device('name': IonQ Device, 'arn': arn:aws:braket:::device/qpu/ionq/ionQdevice)


# Setup Circuit to Execute on QPU

In [14]:
from braket.circuits import Circuit

circuit = Circuit()

a = circuit.h(0) # Put into superposition
b = a.cnot(0, 1) # Entangle a and b

In [15]:
print(b)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


# Run Task on Device

In [40]:
task = device.run(b, s3_output_prefix, shots=100)

In [41]:
print(task)

AwsQuantumTask('id':arn:aws:braket:us-east-1:835319576252:quantum-task/077d816f-dfa8-4f8e-a1e4-7146a37f06ae)


In [42]:
task_arn = task.id

In [43]:
print(task_arn)

arn:aws:braket:us-east-1:835319576252:quantum-task/077d816f-dfa8-4f8e-a1e4-7146a37f06ae


In [44]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/braket/home?region={}#/tasks/">Quantum Tasks</a></b>'.format(region)))


In [45]:
result = task.result().measurement_counts

In [46]:
print(result)

Counter({'11': 54, '00': 45, '10': 1})
